# Imports

In [ ]:
from __future__ import annotations
import hotspot_utils
from hotspot_classes import Threshold, Hotspot

import numpy as np
import pandas as pd
import multiprocessing
from multiprocessing import Pool
from itertools import repeat

import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
from matplotlib.colors import ListedColormap,LinearSegmentedColormap
import matplotlib.patches as mpatches
from matplotlib import colors as mcolors

NUM_CORES = multiprocessing.cpu_count()

import warnings
warnings.filterwarnings("ignore")

# Read in Data

In [ ]:
# This cell reads in parameters and response data from Excel files and combines them into a single dataframe
# Check cell outputs to make sure everything looks good

parameters_file = "Multi-Threshold Analysis Data" # Excel file to pull parameters from
parameters_sheet = "Suzuki Yields and Parameters" # Sheet in the Excel file to pull parameters from
parameters_start_col = 3   # 0-indexed column number where the parameters start
parameters_num_parameters = 190 # Number of parameters in the parameters file
parameters_num_responses = 450 # Number of responses/ligands in the parameters file
parameters_y_label_col = 0  # 0-indexed column number where the ligand labels are
parameters_header_rows = 0 # Number of rows to skip when reading the parameters

response_file = "Multi-Threshold Analysis Data" # Excel file to pull responses from
response_sheet = "Suzuki Yields and Parameters" # Sheet in the Excel file to pull responses from
response_num_samples = 450 # Number of samples/reactions in the response file
response_col = 1 # 0-indexed column number for the responses
response_y_label_col = 0  # 0-indexed column number where the ligand labels are
response_header_rows = 1 # Number of rows to skip when reading the responses

# If the parameters and outputs are on different sheets, you can use the following code to read them in separately

# parameters_file = "kraken descriptors" # Excel file to pull parameters from
# parameters_sheet = "DFT_data" # Sheet in the Excel file to pull parameters from
# parameters_start_col = 1   # 0-indexed column number where the parameters start
# parameters_num_parameters = 190 # Number of parameters in the parameters file
# parameters_num_responses = 1560 # Number of responses/ligands in the parameters file
# parameters_y_label_col = 0  # 0-indexed column number where the ligand labels are
# parameters_header_rows = 0 # Number of rows to skip when reading the parameters

# response_file = "Multi-Threshold Analysis Data" # Excel file to pull responses from
# response_sheet = "Reactions VI-IX" # Sheet in the Excel file to pull responses from
# response_num_samples = 100 # Number of samples/reactions in the response file
# response_col = 2 # 0-indexed column number for the responses
# response_y_label_col = 0  # 0-indexed column number where the ligand labels are
# response_header_rows = 0 # Number of rows to skip when reading the responses

# --------------------------------------------------------------------------------------------------------------
# EDIT ABOVE THIS LINE
# --------------------------------------------------------------------------------------------------------------

# Actually start reading stuff into dataframes
parameters_df = pd.read_excel("./InputData/" + parameters_file + ".xlsx",
                              parameters_sheet,
                              header = parameters_header_rows,
                              index_col = parameters_y_label_col,
                              nrows = parameters_num_responses + 1,
                              usecols = list(range(0, (parameters_num_parameters + parameters_start_col)))
                              )
response_df = pd.read_excel("./InputData/" + response_file + ".xlsx",
                            response_sheet,
                            header = response_header_rows,
                            index_col = response_y_label_col,
                            nrows = response_num_samples,
                            usecols = list(range(0, response_col + 1))
                            )


# Drop any columns before parameters_start_col that are not the index column
parameters_columns_to_keep = [col for col in range(0, len(parameters_df.columns)) if col >= parameters_start_col-1]
parameters_df = parameters_df.iloc[:,parameters_columns_to_keep]

# Combine the two dataframes into the master dataframe
response_df.drop(response_df.columns[0:response_col-1], axis = 'columns', inplace = True)
response_df['y_class'] = 0 # This should create the "y_class" column that will ultimately be used for classification labels
data_df = response_df.merge(parameters_df, left_index = True, right_index = True)
data_df.columns.values[0] = 'response' # Converts the output column name from whatever it is on the spreadsheet
data_df.dropna(inplace = True) # This covers the entire masking section and trims the dataframe down to only the rows relevant to this dataset

# This converts all the data to numeric values since it was reading them in as non-numeric objects for some reason
for column in data_df.columns:
    data_df[column] = pd.to_numeric(data_df[column], errors='coerce')
data_df.dropna(inplace = True)

# Creates a dictionary to convert x# labels to full parameter names
x_names = list(parameters_df.iloc[0, :parameters_num_parameters])
x_labels = list(parameters_df.columns)[:parameters_num_parameters]
x_labelname_dict = dict(zip(x_labels, x_names))

# Print out some information about the dataframe to confirm it was read in correctly
print("Parameter file shape: {}".format(parameters_df.shape))
print("Final parameter quantity: {}".format(len(x_labels)))
print("Final experiment quantity: {}".format(data_df.shape[0]))
print("First parameter cell: {}".format(data_df[x_labels[0]].iloc[0]))
print("Last parameter cell:  {}".format(data_df[x_labels[-1]].iloc[-1]))
print("First response: {}".format(data_df.iloc[0,0]))
print("Last response:  {}".format(data_df.iloc[-1,0]))
print("First reaction label: {}".format(data_df.index[0]))
print("Last reaction label:  {}".format(data_df.index[-1]))

display(data_df)

# Training/Test Split

In [ ]:
# Divide the data into training and test sets
# split options are 'random', 'ks' (kennard-stone), 'y_equidist' (equidistant in y), 'define' (manual selection) and 'none'
split = "none"
test_ratio = 0.3 

training_set, test_set = hotspot_utils.train_test_splits(data_df, split, test_ratio)

# Run the Combined Threshold Finder

## Automatic Threshold Analysis

Edit and run the first cell to set the parameters of the threshold analysis, then run the second cell without changing anything (unless you have a good reason to) to start the calculations.

This section automatically identifies the best hotspots/multi-thresholds based on the parameters set in the first cell.

In [ ]:
# Cutoff in your output for what counts as an active ligand
y_cut = 10

# Set to True if you want points below the y-cut to be considered active
low_is_good = False

# How heavily to value active ligands (1) over inactive ligands (0)
class_weight = {1:10, 0:1} 

# How the prune_hotspots and find_best_hotspots evaluates which are the best
# Can be set to 'accuracy', 'weighted_accuracy', 'f1', and 'weighted_f1'
evaluation_method = 'weighted_accuracy'

# How many threshold dimensions do you want?
n_thresholds = 2

# What percentage of thresholds are analyzed in each subsequent step
percentage = 100

In [ ]:
# Set up y_class, the binary list of which y values are above y_cut
if(low_is_good):
    for i in data_df.index:
        data_df.loc[i, 'y_class'] = int(data_df.loc[i, 'response'] < y_cut)
else:
    for i in data_df.index:
        data_df.loc[i, 'y_class'] = int(data_df.loc[i, 'response'] > y_cut)

# Find the best thresholds within the full X and y space and make single threshold hotspot objects from them
all_thresholds = hotspot_utils.threshold_generation(data_df, class_weight, evaluation_method, x_labelname_dict)
best_hotspots = []
for thresh in all_thresholds:
    temp_hs = Hotspot(data_df, [thresh], y_cut, training_set, test_set, evaluation_method, class_weight)
    best_hotspots.append(temp_hs)

# Cut down to the best {percentage} hotspots
best_hotspots = hotspot_utils.prune_hotspots(best_hotspots, percentage, evaluation_method)

# Add more thresholds, pruning after each step for resource management
for i in range(n_thresholds - 1):
    with Pool(processes=int(NUM_CORES*.7)) as p:
        new_hotspots = p.starmap(hotspot_utils.hs_next_thresholds_fast, zip(best_hotspots, repeat(all_thresholds)))
    new_hotspots = [item for sublist in new_hotspots for item in sublist] 
    
    best_hotspots = hotspot_utils.prune_hotspots(new_hotspots, percentage, evaluation_method)

best_hotspots.sort(key = lambda x: x.accuracy_dict[evaluation_method], reverse = True)

# print the top 5 hotspots
for i, hs in enumerate(best_hotspots[:5]):
    print(f'Hotspot Index: {i}')
    print(hs)
    hs.print_stats()
    print('\n**********************************\n')

## Manual Threshold Selection

Edit and run the first cell to set the parameters of the threshold analysis, then run the second cell without changing anything (unless you have a good reason to) to start the calculations.

This section allows you to manually select which feature(s) or range of features are used in the analysis.

In [ ]:
# What features do you want in your hotspot?
# For ranges, use data_df.columns[x:y].to_list() to get a list of feature xIDs
# For specific features, use ['xID1', 'xID2', ...]

manual_features = [['x1'], ['x87']]
# manual_features = [['x87']]
# manual_features = [['x1'], ['x87'], data_df.columns[3:].to_list()]

# Cutoff for what counts as a hit
y_cut = 10

# How heavily to value hits (1) over misses (0)
class_weight = {1:10, 0:1} 

# What percentage of hotspots to take through to each subsequent step
# Only relevant if using ranges instead of specific parameters
percentage = 100

# How the prune_hotspots and find_best_hotspots evaluates which are the best
# Can be set to 'accuracy', 'weighted_accuracy', 'f1', and 'weighted_f1'
evaluation_method = 'weighted_accuracy'

# Set to True if you want a hotspot of low output results (cold spot?)
low_is_good = False

In [ ]:
# Set up y_class, the binary list of which y values are above y_cut
if(low_is_good):
    for i in data_df.index:
        data_df.loc[i, 'y_class'] = int(data_df.loc[i, 'response'] < y_cut)
else:
    for i in data_df.index:
        data_df.loc[i, 'y_class'] = int(data_df.loc[i, 'response'] > y_cut)

# Find the best thresholds within the full X and y space and make single threshold hotspot objects from them
all_thresholds = hotspot_utils.threshold_generation(data_df, class_weight, evaluation_method, x_labelname_dict, manual_features[0])
best_hotspots = []
for thresh in all_thresholds:
    temp_hs = Hotspot(data_df, [thresh], y_cut, training_set, test_set, evaluation_method, class_weight)
    best_hotspots.append(temp_hs)

# Cut down to the best {percentage} hotspots
best_hotspots = hotspot_utils.prune_hotspots(best_hotspots, percentage, evaluation_method)

# Add more thresholds, pruning after each step for resource management
for i in range(len(manual_features) - 1):
    new_hotspots = []
    for hs in best_hotspots:
        temp_hotspots = hotspot_utils.hs_next_thresholds(hs, data_df, class_weight, x_labelname_dict, manual_features[i+1])
        new_hotspots.extend(temp_hotspots)
    best_hotspots = new_hotspots
    del (new_hotspots)
    best_hotspots = hotspot_utils.prune_hotspots(best_hotspots, percentage, evaluation_method)
    
best_hotspots.sort(key = lambda x: x.accuracy_dict[evaluation_method], reverse = True)

# print the top 5 hotspots
for i, hs in enumerate(best_hotspots[:5]):
    print(f'Hotspot Index: {i}')
    print(hs)
    hs.print_stats()
    print('\n**********************************\n')

# Analysis

## Print Longer List of Hotspots

This cell prints the text output for a longer list of hotspots if you want to see more than were given in the previous sections.

In [ ]:
# How many hotspots do you want listed?
n = 50

for i, hs in enumerate(best_hotspots[:n]):
    print(f'Hotspot Index: {i}')
    print(hs)
    hs.print_stats()
    print('\n**********************************\n')

## Display Individual Hotspot

### Plotting Functions

Run these functions before running the Display Plots section.

In [ ]:
# This is a set of functions for plotting multi-threshold hotspots. They've been included in the main script for ease of modification if you want to change how the plots look.

def plot(hs:Hotspot, subset:str = 'all', coloring:str = 'scaled', gradient_color:str = 'Oranges', output_label:str = '% Yield'):
    """
    Plot a single, double, or triple threshold by calling the relevant function

    :hs: Hotspot object to plot
    :subset: 'all', 'train', or 'test'; indicates which subset to show on the plot
    :coloring: 'scaled' or 'binary'; indicates if points should be colored based on actual output values or by output category
    :gradient_color: the color scheme applied to the heatmap, default 'Oranges'
    :output_label: default '% Yield'
    """
    if(len(hs.thresholds)==1):
        plot_single_threshold(hs, subset, coloring, gradient_color, output_label)
    elif(len(hs.thresholds)==2):
        plot_double_threshold(hs, subset, coloring, gradient_color, output_label)
    elif(len(hs.thresholds)==3):
        plot_triple_threshold(hs, subset, coloring, gradient_color, output_label)
    else:
        print(f'Unable to plot {len(hs.thresholds)} thresholds')

def plot_single_threshold(hs:Hotspot, subset:str = 'all', coloring:str = 'scaled', gradient_color:str = 'Oranges', output_label:str = '% Yield'):
    """
    Plot a single threshold in 2 dimensions

    :hs: Hotspot object to plot
    :subset: 'all', 'train', or 'test'; indicates which subset to show on the plot
    :coloring: 'scaled' or 'binary'; indicates if points should be colored based on actual output values or by output category
    :gradient_color: the color scheme applied to the heatmap, default 'Oranges'
    :output_label: default '% Yield'
    """

    x_col = hs.thresholds[0].index
    plt.figure(figsize=(12, 8))

    # This section auto-scales the plot
    x_min = float(min(hs.data_df.loc[:, x_col]))
    x_max = float(max(hs.data_df.loc[:, x_col]))
    y_min = float(min(hs.data_df.loc[:, 'response']))
    y_max = float(max(hs.data_df.loc[:, 'response']))

    dx = abs(x_min - x_max)
    dy = abs(y_min - y_max)

    x_min = x_min - abs(dx * 0.05)
    x_max = x_max + abs(dx * 0.05)
    y_min = y_min - abs(dy * 0.05)
    y_max = y_max + abs(dy * 0.05)
    
    # Set which points to plot based on the subset parameter
    if(subset == 'all'):
        points_to_plot = hs.data_df.index
    elif(subset == 'train'):
        points_to_plot = hs.training_set
    elif(subset == 'test'):
        points_to_plot = hs.test_set
    else:
        points_to_plot = []
    
    # Change how the points are colored, controlled by the coloring parameter
    if(coloring=='scaled'):
        mapping_cl = hs.data_df.loc[points_to_plot, 'response']
    elif(coloring=='binary'):
        mapping_cl = hs.data_df.loc[points_to_plot, 'y_class']
    else:
        mapping_cl = hs.data_df.loc[points_to_plot, 'response']

    # This is where it selects which descriptors to plot
    x = hs.data_df.loc[points_to_plot, x_col]
    y = hs.data_df.loc[points_to_plot, 'response']
    plt.scatter(x, y, c = mapping_cl, cmap = gradient_color, edgecolor ='black', s = 100)
    
    # Set the gradient bar or binary legend
    if(coloring == 'scaled'):
        cbar = plt.colorbar(plt.gca().collections[0], shrink=1)
        cbar.set_label(output_label, rotation=90, size=25)     
        cbar.ax.tick_params(labelsize=18)   
    elif(coloring == 'binary'):
        colormap = plt.get_cmap(gradient_color)
        active_color = mcolors.to_hex(colormap(1.0))
        inactive_color = mcolors.to_hex(colormap(0.0))
        active_patch = mpatches.Patch(color=active_color, label='Active ligands', edgecolor='black')
        inactive_patch = mpatches.Patch(color=inactive_color, label='Inactive ligands', edgecolor='black')
        plt.legend(handles=[active_patch, inactive_patch], fontsize=15, loc='upper right', edgecolor='black')
 
    
    # Draw the threshold line
    plt.axvline(x=hs.thresholds[0].cut_value, color='black', linestyle='--')
    # Draw y_cut line
    plt.axhline(y=hs.y_cut, color='r', linestyle='--')

    # Axis setup
    plt.xlabel(f'{hs.thresholds[0].feature_label} {hs.thresholds[0].feature_name}',fontsize=25)
    plt.ylabel(output_label,fontsize=25)
    plt.xticks(fontsize=18)
    plt.xlim(x_min, x_max)
    plt.locator_params(axis='x', nbins=5)
    plt.yticks(fontsize=18)
    plt.ylim(y_min, y_max)
    plt.locator_params(axis='y', nbins=4)

    plt.title(f'{hs.thresholds[0].feature_name} Threshold', fontsize = 25)
    
    plt.show()

def plot_double_threshold(hs:Hotspot, subset:str = 'all', coloring:str = 'scaled', gradient_color:str = 'Oranges', output_label:str = 'Yield (%)'):
    """
    Plot a double threshold in 2 dimensions

    :subset: 'all', 'train', or 'test'; indicates which subset to show on the plot
    :coloring: 'scaled' or 'binary'; indicates if points should be colored based on actual output values or by output category
    :gradient_color: the color scheme applied to the heatmap, default 'Oranges'
    :output_label: default 'Yield (%)'
    """

    x_col,y_col = hs.thresholds[0].index, hs.thresholds[1].index
    plt.figure(figsize=(12, 8))

    # This section auto-scales the plot
    x_min = float(min(hs.data_df.loc[:, x_col]))
    x_max = float(max(hs.data_df.loc[:, x_col]))
    y_min = float(min(hs.data_df.loc[:, y_col]))
    y_max = float(max(hs.data_df.loc[:, y_col]))

    dx = abs(x_min - x_max)
    dy = abs(y_min - y_max)

    x_min = x_min - abs(dx * 0.05)
    x_max = x_max + abs(dx * 0.05)
    y_min = y_min - abs(dy * 0.05)
    y_max = y_max + abs(dy * 0.05)
    
    # Set which points to plot based on the subset parameter
    if(subset == 'all'):
        points_to_plot = hs.data_df.index
    elif(subset == 'train'):
        points_to_plot = hs.training_set
    elif(subset == 'test'):
        points_to_plot = hs.test_set
    else:
        points_to_plot = []
    
    # Change how the points are colored, controlled by the coloring parameter
    if(coloring=='scaled'):
        mapping_cl = hs.data_df.loc[points_to_plot, 'response']
    elif(coloring=='binary'):
        mapping_cl = hs.data_df.loc[points_to_plot, 'y_class']
    else:
        mapping_cl = hs.data_df.loc[points_to_plot, 'response']

    # This is where it selects which descriptors to plot
    x = hs.data_df.loc[points_to_plot,x_col]
    y = hs.data_df.loc[points_to_plot,y_col]
    plt.scatter(x, y, c=mapping_cl,cmap=gradient_color, edgecolor='black', s=100)  

    # Draw threshold lines
    plt.axhline(y=hs.thresholds[1].cut_value, color='black', linestyle='--')
    plt.axvline(x=hs.thresholds[0].cut_value, color='black', linestyle='--')
    
    # Set the gradient bar or binary legend
    if(coloring == 'scaled'):
        cbar = plt.colorbar(plt.gca().collections[0], shrink=1)
        cbar.set_label(output_label, rotation=90, size=18)
    elif(coloring == 'binary'):
        colormap = plt.get_cmap(gradient_color)
        active_color = mcolors.to_hex(colormap(1.0))
        inactive_color = mcolors.to_hex(colormap(0.0))
        active_patch = mpatches.Patch(color=active_color, label='Active ligands', edgecolor='black')
        inactive_patch = mpatches.Patch(color=inactive_color, label='Inactive ligands', edgecolor='black')
        plt.legend(handles=[active_patch, inactive_patch], fontsize=15, loc='upper right', edgecolor='black')

    # Axis setup
    plt.xlabel(f'{hs.thresholds[0].feature_label} {hs.thresholds[0].feature_name}', fontsize = 15)
    plt.ylabel(f'{hs.thresholds[1].feature_label} {hs.thresholds[1].feature_name}', fontsize = 15)
    plt.xticks(fontsize = 18)
    plt.xlim(x_min, x_max)
    plt.locator_params(axis = 'x', nbins = 5)
    plt.yticks(fontsize = 18)
    plt.ylim(y_min, y_max)
    plt.locator_params(axis = 'y', nbins = 4)

    # Print the title of the plot
    plt.title(f'{hs.thresholds[0].feature_name} x {hs.thresholds[1].feature_name}', fontsize = 20)

    plt.show()

def plot_triple_threshold(hs:Hotspot, subset:str ='all', coloring:str = 'scaled', gradient_color:str = 'Oranges', output_label:str = '% Yield'):
    """
    Plot a triple threshold in 3 dimensions

    :subset: 'all', 'train', or 'test'; indicates which subset to show on the plot
    :coloring: 'scaled' or 'binary'; indicates if points should be colored based on actual output values or by output category
    :gradient_color: the color scheme applied to the heatmap, default 'Oranges'
    :output_label: default '% Yield'
    """

    x_col,y_col,z_col = hs.thresholds[0].index, hs.thresholds[1].index, hs.thresholds[2].index
    
    # Auto-scale the plot to your data
    x_min = float(min(hs.data_df.loc[:, x_col]))
    x_max = float(max(hs.data_df.loc[:, x_col]))
    y_min = float(min(hs.data_df.loc[:, y_col]))
    y_max = float(max(hs.data_df.loc[:, y_col]))
    z_min = float(min(hs.data_df.loc[:, z_col]))
    z_max = float(max(hs.data_df.loc[:, z_col]))

    dx = abs(x_min - x_max)
    dy = abs(y_min - y_max)
    dz = abs(z_min - z_max)

    x_min = x_min - abs(dx * 0.05)
    x_max = x_max + abs(dx * 0.05)
    y_min = y_min - abs(dy * 0.05)
    y_max = y_max + abs(dy * 0.05)
    z_min = z_min - abs(dz * 0.05)
    z_max = z_max + abs(dz * 0.05)

    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(111, projection = '3d')

    # Set which points to plot based on the subset parameter
    if(subset == 'all'):
        points_to_plot = hs.data_df.index
    elif(subset == 'train'):
        points_to_plot = hs.training_set
    elif(subset == 'test'):
        points_to_plot = hs.test_set
    else:
        points_to_plot = []
        
    # Change how the points are colored, controlled by the coloring parameter
    if(coloring=='scaled'):
        mapping_cl = hs.data_df.loc[points_to_plot, 'response']
    elif(coloring=='binary'):
        mapping_cl = hs.data_df.loc[points_to_plot, 'y_class']
    else:
        mapping_cl = hs.data_df.loc[points_to_plot, 'response']

    # Actually plot the data
    x = hs.data_df.loc[points_to_plot,x_col]
    y = hs.data_df.loc[points_to_plot,y_col]
    z = hs.data_df.loc[points_to_plot,z_col]
    ax.scatter(x, y, z, c=mapping_cl, cmap=gradient_color, alpha=0.95, marker="s", s=50, edgecolors='k') 
        
    # Plot the z-axis threshold
    temp_x = np.linspace(x_min, x_max, num=10)
    temp_y = np.linspace(y_min, y_max, num=10)
    temp_x, temp_y = np.meshgrid(temp_x, temp_y)
    temp_z = hs.thresholds[2].cut_value + 0 * temp_x + 0 * temp_y
    ax.plot_surface(temp_x, temp_y, temp_z, alpha=0.15, color='gray')
    
    # Plot the x-axis threshold
    temp_y = np.linspace(y_min, y_max, num=10)
    temp_z = np.linspace(z_min, z_max, num=10)
    temp_z, temp_y = np.meshgrid(temp_z, temp_y)
    temp_x = hs.thresholds[0].cut_value + 0 * temp_z + 0 * temp_y
    ax.plot_surface(temp_x, temp_y, temp_z, alpha=0.15, color='gray') 
    
    # Plot the y-axis threshold
    temp_x = np.linspace(x_min, x_max, num = 10)
    temp_z = np.linspace(z_min, z_max, num = 10)
    temp_x, temp_z = np.meshgrid(temp_x, temp_z)
    temp_y = hs.thresholds[1].cut_value + 0 * temp_x + 0 * temp_z
    ax.plot_surface(temp_x, temp_y, temp_z, alpha=0.15, color='gray')
    
    plt.xticks(fontsize = 10) 
    plt.yticks(fontsize = 10)

    # Set axes labels
    ax.set_xlabel(f'{hs.thresholds[0].feature_label} {hs.thresholds[0].feature_name}',fontsize=12.5)
    ax.set_ylabel(f'{hs.thresholds[1].feature_label} {hs.thresholds[1].feature_name}',fontsize=12.5)
    ax.set_zlabel(f'{hs.thresholds[2].feature_label} {hs.thresholds[2].feature_name}',fontsize=12.5)
    plt.locator_params(axis='y', nbins=8)

    ax.set_xlim(x_min, x_max)
    ax.set_ylim(y_min, y_max)
    ax.set_zlim(z_min, z_max)
    
    # Set the gradient bar on the side
    if(coloring == 'scaled'):
        cbar = plt.colorbar(plt.gca().collections[0], shrink=0.5)
        cbar.set_label(output_label, rotation=90, size=18)

    plt.show()

### Display Plots

For more direct control over plot style, changes can be made to functions in the cell above. The above functions must be run before this cell can be run.

In [ ]:
# Select the hotspot you want to plot based on its index
hotspot_index = 0
print(best_hotspots[hotspot_index])


# subset can be 'all', 'training', or 'test'
# You can change the coloring to either 'scaled' or 'binary'
# output_label is whatever you call your output (Only relevant when using 'scaled' coloring or single threshold)
plot(best_hotspots[hotspot_index], subset='all', coloring='binary', output_label='Yield (%)', gradient_color='Oranges')

# Virtual Screening / Validation

## Import Virtual Screening / Validation Parameter Data

In [ ]:
# Import the parameters for the previously unseen molecules, required for both virtual screening and validation
# This assumes the excel sheet has a row of xID labels then a row of full parameter names
parameters_file = "kraken descriptors" 
parameters_sheet = "DFT_data" 
parameters_num_parameters = 190
parameters_start_col = 1   # 0-indexed
parameters_num_samples = 1560 
parameters_y_label_col = 0  # 0-indexed
parameters_header_rows = 0


# Actually start reading stuff into the parameter dataframe
vs_parameters_df = pd.read_excel("./InputData/" + parameters_file + ".xlsx",
                              parameters_sheet,
                              header = parameters_header_rows,
                              index_col = parameters_y_label_col,
                              nrows = parameters_num_samples + 1,
                              usecols = list(range(0, (parameters_num_parameters + parameters_start_col)))
                              )

# Create a dictionary to convert x# labels to full parameter names
vs_x_names = list(vs_parameters_df.iloc[0, parameters_start_col - 1 : parameters_num_parameters + parameters_start_col - 1])
vs_x_labels = list(vs_parameters_df.columns)[parameters_start_col - 1 : parameters_num_parameters + parameters_start_col - 1]
vs_x_labelname_dict = dict(zip(vs_x_labels, vs_x_names))

vs_parameters_df.drop(vs_parameters_df.index[0], inplace=True) # Remove the full parameter name row
vs_parameters_df.index = vs_parameters_df.index.astype(int) # This converts the index (molecule/reaction number) from strings to ints

# Convert all the data to numeric values since it was reading them in as non-numeric objects for some reason
for column in vs_parameters_df.columns:
    vs_parameters_df[column] = pd.to_numeric(vs_parameters_df[column], errors='coerce')

display(vs_parameters_df)

## Validation

In [ ]:
# Read the validation results from the excel sheet
# The final result should be a dataframe with indicies corresponding to the vs_parameters_df from above,
# a column of experimental outputs, and all the relevant parameters

validation_file = "Multi-Threshold Analysis Data"
validation_sheet = "Reaction XII"
validation_experimental_col = 2

############################################################################################################

vs_response_df = pd.read_excel('./InputData/' + validation_file + '.xlsx',
                              sheet_name=validation_sheet,
                              index_col=0,
                              )

# Drop all columns except the experimental output
vs_response_df = vs_response_df.iloc[:, [validation_experimental_col-1]]
vs_response_df.columns = ['response']

for column in vs_response_df.columns:
    vs_response_df[column] = pd.to_numeric(vs_response_df[column], errors='coerce')
vs_response_df.dropna(inplace = True)

vs_combined_df = pd.concat([vs_response_df, vs_parameters_df], axis=1, join='inner')

display(vs_combined_df)

### Print/Export Validation Classifications

In [ ]:
# Select hotspot to validate
hotspot_index = 0

# Set to True if you want to write the validation results to an Excel file
write_to_excel = False
output_name = 'validation_output.xlsx'

############################################################################################################

hs = best_hotspots[hotspot_index]

# Begin the output dataframe with the experimental results and the y_class column
validation_report_df = vs_response_df.copy()
validation_report_df['y_class'] = [y > hs.y_cut for y in validation_report_df.iloc[:, 0]]

# Add the parameter values and binary evaluations for each ligand to the output dataframe
# True in an xID column indicates that the ligand is predicted to be active by that threshold
for parameter in hs.threshold_indexes:
    validation_report_df[f'{parameter}_{vs_x_labelname_dict[parameter]}'] = vs_parameters_df.loc[validation_report_df.index, parameter]

threshold_evaluations = hs.expand(vs_parameters_df.loc[validation_report_df.index])
validation_report_df = pd.concat([validation_report_df, threshold_evaluations], axis=1)

# Create a column for the final hotspot evaluation if there is more than a single threshold
if len(hs.thresholds) > 1:
    for ligand in validation_report_df.index:
        evaluation = all([validation_report_df.loc[ligand, threshold] for threshold in hs.threshold_indexes])
        validation_report_df.loc[ligand, 'Full Hotspot Evaluation'] = evaluation

display(validation_report_df)

hs.get_external_accuracy(vs_combined_df, verbose=True)

if write_to_excel:
    validation_report_df.to_excel(output_name)

### Plot Validation Results

In [ ]:
# Plot your validation data on the hotspot selected by hotspot_index
# Modify the last line for plotting options
hotspot_index = 0
hs = best_hotspots[hotspot_index]

def plot_validation(hs:Hotspot, response_data:pd.DataFrame, vs_parameters:pd.DataFrame, subset:str = 'all', coloring:str = 'scaled', gradient_color:str = 'Oranges', output_label:str = 'Yield (%)', hide_og=False):
    """
    Plot a single, double, or triple threshold by calling the relevant function

    :hs: The hotspot object to plot
    :response_data: The experimental output data for the validation set
    :vs_parameters: The parameter data for the validation set
    :subset: 'all', 'train', or 'test'; indicates which subset to show on the plot if also plotting the original data
    :coloring: 'scaled' or 'binary'; indicates if points should be colored based on actual output values or by output category
    :gradient_color: the color scheme applied to the heatmap, default 'Oranges'
    :output_label: default 'Yield (%)'
    :hide_og: True (default) if you want to hide the original data points
    """
    if(len(hs.thresholds)==1):
        plot_validation_single_threshold(hs, response_data, vs_parameters, subset, coloring, gradient_color, output_label, hide_og)
    elif(len(hs.thresholds)==2):
        plot_validation_double_threshold(hs, response_data, vs_parameters, subset, coloring, gradient_color, output_label, hide_og)
    elif(len(hs.thresholds)==3):
        plot_validation_triple_threshold(hs, response_data, vs_parameters, subset, coloring, gradient_color, output_label, hide_og)
    else:
        print(f'Unable to plot {len(hs.thresholds)} thresholds')

def plot_validation_single_threshold(hs:Hotspot, response_data:pd.DataFrame, vs_parameters:pd.DataFrame, subset:str = 'all', coloring:str = 'scaled', gradient_color:str = 'Oranges', output_label:str = 'Yield (%)', hide_og=False):
    """
    Plot a validation set on a single threshold

    :hs: The hotspot object to plot
    :response_data: The experimental output data for the validation set
    :vs_parameters: The parameter data for the validation set
    :subset: 'all', 'train', or 'test'; indicates which subset to show on the plot if also plotting the original data
    :coloring: 'scaled' or 'binary'; indicates if points should be colored based on actual output values or by output category
    :gradient_color: the color scheme applied to the heatmap, default 'Oranges'
    :output_label: default 'Yield (%)'
    :hide_og: True (default) if you want to hide the original data points
    """

    x_col = hs.thresholds[0].index
    plt.figure(figsize=(12, 8))

    # This section auto-scales the plot
    x_min = float(min(hs.data_df.loc[:, x_col]))
    x_max = float(max(hs.data_df.loc[:, x_col]))
    y_min = float(min(hs.data_df.loc[:, 'response']))
    y_max = float(max(hs.data_df.loc[:, 'response']))

    dx = abs(x_min - x_max)
    dy = abs(y_min - y_max)

    x_min = x_min - abs(dx * 0.05)
    x_max = x_max + abs(dx * 0.05)
    y_min = y_min - abs(dy * 0.05)
    y_max = y_max + abs(dy * 0.05)
    
    # Set which points to plot based on the subset parameter
    if(subset == 'all'):
        points_to_plot = hs.data_df.index
    elif(subset == 'train'):
        points_to_plot = hs.training_set
    elif(subset == 'test'):
        points_to_plot = hs.test_set
    else:
        points_to_plot = []
    
    # Change how the points are colored, controlled by the coloring parameter
    if(coloring=='scaled'):
        mapping_cl = hs.data_df.loc[points_to_plot, 'response']
        vs_mapping_cl = response_data.iloc[:, 0]
    elif(coloring=='binary'):
        mapping_cl = hs.data_df.loc[points_to_plot, 'y_class']
        vs_mapping_cl = [1 if i >= hs.y_cut else 0 for i in response_data.iloc[:, 0]]
    else:
        mapping_cl = hs.data_df.loc[points_to_plot, 'response']

    # This is where it selects which descriptors to plot
    if not hide_og:
        x = hs.data_df.loc[points_to_plot, x_col]
        y = hs.data_df.loc[points_to_plot, 'response']
        plt.scatter(x, y, c = mapping_cl, cmap = gradient_color, edgecolor ='black', s = 100)

    # Plot the validation data set on top of the hotspot
    vs_x = vs_parameters.loc[response_data.index, x_col]
    vs_y = response_data.iloc[:, 0]
    plt.scatter(vs_x, vs_y, c = vs_mapping_cl, cmap = gradient_color, edgecolor = 'black', s = 100, marker = 's')
    
    # Set the gradient bar or binary legend
    if(coloring == 'scaled'):
        cbar = plt.colorbar(plt.gca().collections[0], shrink=1)
        cbar.set_label(output_label, rotation=90, size=25)     
        cbar.ax.tick_params(labelsize=18)   
    elif(coloring == 'binary'):
        colormap = plt.get_cmap(gradient_color)
        active_color = mcolors.to_hex(colormap(1.0))
        inactive_color = mcolors.to_hex(colormap(0.0))
        active_patch = mpatches.Patch(color=active_color, label='Active ligands', edgecolor='black')
        inactive_patch = mpatches.Patch(color=inactive_color, label='Inactive ligands', edgecolor='black')
        plt.legend(handles=[active_patch, inactive_patch], fontsize=15, loc='upper right', edgecolor='black')
 
    
    # Draw the threshold line
    plt.axvline(x=hs.thresholds[0].cut_value, color='black', linestyle='--')
    # Draw y_cut line
    plt.axhline(y=hs.y_cut, color='r', linestyle='--')

    # Axis setup
    plt.xlabel(f'{hs.thresholds[0].feature_label} {hs.thresholds[0].feature_name}',fontsize=25)
    plt.ylabel(output_label,fontsize=25)
    plt.xticks(fontsize=18)
    plt.xlim(x_min, x_max)
    plt.locator_params(axis='x', nbins=5)
    plt.yticks(fontsize=18)
    plt.ylim(y_min, y_max)
    plt.locator_params(axis='y', nbins=4)

    plt.title(f'{hs.thresholds[0].feature_name} Threshold', fontsize = 25)
    
    plt.show()

    # Calculate the accuracy, precision, and recall of the hotspot
    hs.get_external_accuracy(vs_combined_df, verbose=True)

def plot_validation_double_threshold(hs:Hotspot, response_data:pd.DataFrame, vs_parameters:pd.DataFrame, subset:str = 'all', coloring:str = 'scaled', gradient_color:str = 'Oranges', output_label:str = 'Yield (%)', hide_og=False):
    """
    Plot a validation set on a double threshold

    :hs: The hotspot object to plot
    :response_data: The experimental output data for the validation set
    :vs_parameters: The parameter data for the validation set
    :subset: 'all', 'train', or 'test'; indicates which subset to show on the plot if also plotting the original data
    :coloring: 'scaled' or 'binary'; indicates if points should be colored based on actual output values or by output category
    :gradient_color: the color scheme applied to the heatmap, default 'Oranges'
    :output_label: default 'Yield (%)'
    :hide_og: True (default) if you want to hide the original data points
    """

    x_col,y_col = hs.thresholds[0].index, hs.thresholds[1].index
    plt.figure(figsize=(10, 8))

    # This section auto-scales the plot
    x_min = float(min(hs.data_df.loc[:, x_col]))
    x_max = float(max(hs.data_df.loc[:, x_col]))
    y_min = float(min(hs.data_df.loc[:, y_col]))
    y_max = float(max(hs.data_df.loc[:, y_col]))

    dx = abs(x_min - x_max)
    dy = abs(y_min - y_max)

    x_min = x_min - abs(dx * 0.05)
    x_max = x_max + abs(dx * 0.05)
    y_min = y_min - abs(dy * 0.05)
    y_max = y_max + abs(dy * 0.05)
    
    # Set which points to plot based on the subset parameter
    if(subset == 'all'):
        points_to_plot = hs.data_df.index
    elif(subset == 'train'):
        points_to_plot = hs.training_set
    elif(subset == 'test'):
        points_to_plot = hs.test_set
    else:
        points_to_plot = []
    
    # Change how the points are colored, controlled by the coloring parameter
    if(coloring=='scaled'):
        mapping_cl = hs.data_df.loc[points_to_plot, 'response']
        vs_mapping_cl = response_data.iloc[:, 0]
        vmin, vmax = min(hs.data_df.loc[:, 'response']), max(hs.data_df.loc[:, 'response'])
    elif(coloring=='binary'):
        mapping_cl = hs.data_df.loc[points_to_plot, 'y_class']
        vs_mapping_cl = [1 if i >= hs.y_cut else 0 for i in response_data.iloc[:, 0]]
        vmin, vmax = 0, 1
    else:
        mapping_cl = hs.data_df.loc[points_to_plot, 'response']
        vmin, vmax = min(hs.data_df.loc[:, 'response']), max(hs.data_df.loc[:, 'response'])

    # This is where you choose which descriptors to plot
    if not hide_og:
        x = hs.data_df.loc[points_to_plot,x_col]
        y = hs.data_df.loc[points_to_plot,y_col]
        plt.scatter(x, y, c=mapping_cl,cmap=gradient_color, edgecolor='black', s=100, vmin=vmin, vmax=vmax)  

    # Plot the validation data set on top of the hotspot
    vs_x = vs_parameters.loc[response_data.index, x_col]
    vs_y = vs_parameters.loc[response_data.index, y_col]
    
    colormap = plt.get_cmap(gradient_color)
    plt.scatter(vs_x, vs_y, c=vs_mapping_cl, cmap=gradient_color, edgecolor='black', s=100, marker='s', vmin=vmin, vmax=vmax)
    
    # Draw threshold lines
    plt.axhline(y=hs.thresholds[1].cut_value, color='black', linestyle='--')
    plt.axvline(x=hs.thresholds[0].cut_value, color='black', linestyle='--')
    
    # Set the gradient bar on the side
    if(coloring == 'scaled'):
        cbar = plt.colorbar(plt.gca().collections[1], shrink=1)
        cbar.set_label(output_label, rotation=90, size=18)
        # cbar.set_ticklabels([0,20,40,60,80,100], size=18)
    else:
    # Display a legend
        colormap = plt.get_cmap(gradient_color)
        active_color = mcolors.to_hex(colormap(1.0))
        inactive_color = mcolors.to_hex(colormap(0.0))
        active_patch = mpatches.Patch(color=active_color, label='Active ligands', edgecolor='black')
        inactive_patch = mpatches.Patch(color=inactive_color, label='Inactive ligands', edgecolor='black')
        plt.legend(handles=[active_patch, inactive_patch], fontsize=15, loc='upper right', edgecolor='black')        

    # Axis setup
    plt.xlabel(f'{hs.thresholds[0].feature_label} {hs.thresholds[0].feature_name}', fontsize = 15)
    plt.ylabel(f'{hs.thresholds[1].feature_label} {hs.thresholds[1].feature_name}', fontsize = 15)
    plt.xticks(fontsize = 18)
    plt.xlim(x_min, x_max)
    plt.locator_params(axis = 'x', nbins = 5)
    plt.yticks(fontsize = 18)
    plt.ylim(y_min, y_max)
    plt.locator_params(axis = 'y', nbins = 4)

    # Title the plot
    plt.title(f'{hs.thresholds[0].feature_name} x {hs.thresholds[1].feature_name} Validation', fontsize = 20, y=1.05)

    plt.show()

    # Calculate the accuracy, precision, and recall of the hotspot
    hs.get_external_accuracy(vs_combined_df, verbose=True)

def plot_validation_triple_threshold(hs:Hotspot, response_data:pd.DataFrame, vs_parameters:pd.DataFrame, subset:str = 'all', coloring:str = 'scaled', gradient_color:str = 'Oranges', output_label:str = 'Yield (%)', hide_og=False):
    """
    Plot a validation set on a triple threshold

    :hs: The hotspot object to plot
    :response_data: The experimental output data for the validation set
    :vs_parameters: The parameter data for the validation set
    :subset: 'all', 'train', or 'test'; indicates which subset to show on the plot if also plotting the original data
    :coloring: 'scaled' or 'binary'; indicates if points should be colored based on actual output values or by output category
    :gradient_color: the color scheme applied to the heatmap, default 'Oranges'
    :output_label: default 'Yield (%)'
    :hide_og: True (default) if you want to hide the original data points
    """

    x_col,y_col,z_col = hs.thresholds[0].index, hs.thresholds[1].index, hs.thresholds[2].index

    hotspot_evaluations = hs.expand(vs_parameters_df)
    
    # Auto-scale the plot to your data
    x_min = float(min(hs.data_df.loc[:, x_col]))
    x_max = float(max(hs.data_df.loc[:, x_col]))
    y_min = float(min(hs.data_df.loc[:, y_col]))
    y_max = float(max(hs.data_df.loc[:, y_col]))
    z_min = float(min(hs.data_df.loc[:, z_col]))
    z_max = float(max(hs.data_df.loc[:, z_col]))

    dx = abs(x_min - x_max)
    dy = abs(y_min - y_max)
    dz = abs(z_min - z_max)

    x_min = x_min - abs(dx * 0.05)
    x_max = x_max + abs(dx * 0.05)
    y_min = y_min - abs(dy * 0.05)
    y_max = y_max + abs(dy * 0.05)
    z_min = z_min - abs(dz * 0.05)
    z_max = z_max + abs(dz * 0.05)

    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(111, projection = '3d')

    # Set which points to plot based on the subset parameter
    if(subset == 'all'):
        points_to_plot = hs.data_df.index
    elif(subset == 'train'):
        points_to_plot = hs.training_set
    elif(subset == 'test'):
        points_to_plot = hs.test_set
    else:
        points_to_plot = []
        
    # Change how the points are colored, controlled by the coloring parameter
    if(coloring=='scaled'):
        mapping_cl = hs.data_df.loc[points_to_plot, 'response']
        vs_mapping_cl = response_data.iloc[:, 0]
    elif(coloring=='binary'):
        mapping_cl = hs.data_df.loc[points_to_plot, 'y_class']
        vs_mapping_cl = [1 if i >= hs.y_cut else 0 for i in response_data.iloc[:, 0]]
    else:
        mapping_cl = hs.data_df.loc[points_to_plot, 'response']
        vs_mapping_cl = response_data.iloc[:, 0]

    # Actually plot the data if not hiding the original data
    if(not hide_og):
        x = hs.data_df.loc[points_to_plot,x_col]
        y = hs.data_df.loc[points_to_plot,y_col]
        z = hs.data_df.loc[points_to_plot,z_col]
        ax.scatter(x, y, z, c=mapping_cl, cmap=gradient_color, alpha=0.95, marker="o", s=50, edgecolors='k') 

    # Plot the extra data set on top of the hotspot
    vs_x = vs_parameters.loc[response_data.index, x_col]
    vs_y = vs_parameters.loc[response_data.index, y_col]
    vs_z = vs_parameters.loc[response_data.index, z_col]
    ax.scatter(vs_x, vs_y, vs_z, c=vs_mapping_cl, cmap=gradient_color, edgecolor='black', s=100, marker='s', vmin=0, vmax=1, alpha = 0.95)
        
    # Plot the z-axis threshold
    temp_x = np.linspace(x_min, x_max, num=10)
    temp_y = np.linspace(y_min, y_max, num=10)
    temp_x, temp_y = np.meshgrid(temp_x, temp_y)
    temp_z = hs.thresholds[2].cut_value + 0 * temp_x + 0 * temp_y
    ax.plot_surface(temp_x, temp_y, temp_z, alpha=0.15, color='gray')
    
    # Plot the x-axis threshold
    temp_y = np.linspace(y_min, y_max, num=10)
    temp_z = np.linspace(z_min, z_max, num=10)
    temp_z, temp_y = np.meshgrid(temp_z, temp_y)
    temp_x = hs.thresholds[0].cut_value + 0 * temp_z + 0 * temp_y
    ax.plot_surface(temp_x, temp_y, temp_z, alpha=0.15, color='gray') 
    
    # Plot the y-axis threshold
    temp_x = np.linspace(x_min, x_max, num = 10)
    temp_z = np.linspace(z_min, z_max, num = 10)
    temp_x, temp_z = np.meshgrid(temp_x, temp_z)
    temp_y = hs.thresholds[1].cut_value + 0 * temp_x + 0 * temp_z
    ax.plot_surface(temp_x, temp_y, temp_z, alpha=0.15, color='gray')

    # Draw a line from the point to the base of the plot
    for i in response_data.index.to_list():
        x_point = vs_parameters.loc[i, x_col]
        y_point = vs_parameters.loc[i, y_col]
        z_point = vs_parameters.loc[i, z_col]
        if hotspot_evaluations.loc[i].all() == True:
            color = 'black'
        else:
            color = 'black'

        if hotspot_evaluations.loc[i].all() == True:
            linewidth = 2
            alpha = 0.8
        else:
            linewidth = 1
            alpha = 0.3

        ax.plot([x_point, x_point], [y_point, y_point], [z_point, z_min], color=color, linewidth=linewidth, alpha=alpha)

    
    plt.xticks(fontsize = 10) 
    plt.yticks(fontsize = 10)

    # Set axes labels
    ax.set_xlabel(f'{hs.thresholds[0].feature_label} {hs.thresholds[0].feature_name}',fontsize=15)
    ax.set_ylabel(f'{hs.thresholds[1].feature_label} {hs.thresholds[1].feature_name}',fontsize=15)
    ax.set_zlabel(f'{hs.thresholds[2].feature_label} {hs.thresholds[2].feature_name}',fontsize=15)

    plt.locator_params(axis='y', nbins=8)

    ax.set_xlim(x_min, x_max)
    ax.set_ylim(y_min, y_max)
    ax.set_zlim(z_min, z_max)

   # Set the gradient bar on the side
    if(coloring == 'scaled'):
        cbar = plt.colorbar(plt.gca().collections[0], shrink=0.5)
        cbar.set_label(output_label, rotation=90, size=15)
        cbar.set_ticklabels([0,20,40,60,80,100], size=15)

    # Set initial plot orientation
    ax.view_init(elev=20, azim=210)

    plt.show()

    # Calculate the accuracy, precision, and recall of the hotspot
    hs.get_external_accuracy(vs_combined_df, verbose=True)

plot_validation(hs, vs_response_df, vs_parameters_df, subset='all', coloring='binary', output_label='Yield (%)', hide_og=True)

## Virtual Screening

In [ ]:
# Select hotspot to screen against
hotspot_index = 0

# Set to True if you want to write the virtual screen results to an Excel file
write_to_excel = True
output_name = 'virtual_screen_output.xlsx'

############################################################################################################

hs = best_hotspots[hotspot_index]

# Trim down to the parameter values relevant to this hotspot and add binary evaluations from each ligand
# True in an xID column indicates that the ligand is predicted to be active by that threshold
virtual_screen_report_df = vs_parameters_df.loc[:, hs.threshold_indexes]
virtual_screen_report_df.columns = [f'{parameter}_{vs_x_labelname_dict[parameter]}' for parameter in virtual_screen_report_df.columns]

threshold_evaluations = hs.expand(vs_parameters_df)
virtual_screen_report_df = pd.concat([virtual_screen_report_df, threshold_evaluations], axis=1)

# Create a column for the final hotspot evaluation if there is more than a single threshold
if len(hs.thresholds) > 1:
    for ligand in virtual_screen_report_df.index:
        evaluation = all([virtual_screen_report_df.loc[ligand, threshold] for threshold in hs.threshold_indexes])
        virtual_screen_report_df.loc[ligand, 'Full Hotspot Evaluation'] = evaluation

display(virtual_screen_report_df)

if write_to_excel:
    virtual_screen_report_df.to_excel(output_name)

# Show Hotspots on PCA/UMAP Space

### Read in full descriptor library

In [ ]:
# This assumes the excel sheet has a row of xID labels then a row of full parameter names
# Import the parameters for the virtual screen
dr_parameters_file = "kraken descriptors" 
dr_parameters_sheet = "DFT_data" 
dr_number_parameters = 190
dr_parameters_start_col = 1   # 0-indexed
dr_parameters_num_samples = 1560 
dr_parameters_y_label_col = 0  # 0-indexed
dr_parameters_header_rows = 0


# Actually start reading stuff into the parameter dataframe
dr_parameters_df = pd.read_excel("./InputData/" + dr_parameters_file + ".xlsx",
                              dr_parameters_sheet,
                              header = dr_parameters_header_rows,
                              index_col = dr_parameters_y_label_col,
                              nrows = dr_parameters_num_samples + 1,
                              usecols = list(range(0, (dr_number_parameters + dr_parameters_start_col)))
                              )

dr_parameters_df.drop(dr_parameters_df.index[0], inplace=True) # Remove the full parameter name row
dr_parameters_df.index = dr_parameters_df.index.astype(int) # This converts the index (molecule/reaction number) from strings to ints

# Convert all the data to numeric values since it was reading them in as non-numeric objects for some reason
dr_parameters_df = dr_parameters_df.dropna()
for column in dr_parameters_df.columns:
    dr_parameters_df[column] = pd.to_numeric(dr_parameters_df[column], errors='coerce')

# Create a dictionary to convert x# labels to full parameter names
dr_x_names = list(dr_parameters_df.iloc[0, dr_parameters_start_col - 1 : dr_number_parameters + dr_parameters_start_col - 1])
dr_x_labels = list(dr_parameters_df.columns)[dr_parameters_start_col - 1 : dr_number_parameters + dr_parameters_start_col - 1]
dr_x_labelname_dict = dict(zip(dr_x_labels, dr_x_names))

display(dr_parameters_df)

### Plot the hotspot on reduced dimension space

In [ ]:
# Select which dimensionality reduction type to plot (PCA or UMAP)
DR_sheet = 'UMAP'

plt.figure(figsize=(10, 8))
DR = pd.read_excel('./InputData/kraken descriptors.xlsx', DR_sheet, index_col=0, header=0)

############################################################################################################

hotspot_validation_list = best_hotspots[0].expand(dr_parameters_df)
print(hotspot_validation_list)

# Make a list of indicies in in_hotspot_list where both columns are true
in_hotspot_list = hotspot_validation_list[hotspot_validation_list.all(axis=1)]
in_hotspot_list = in_hotspot_list.index.to_list()

# Make a list of indicies in in_hotspot_list where both columns are not true
not_in_hotspot_list = hotspot_validation_list[~hotspot_validation_list.all(axis=1)]
not_in_hotspot_list = not_in_hotspot_list.index.to_list()
not_in_hotspot_list = [x for x in not_in_hotspot_list if x in DR.index]

x_row = 0
y_row = 1

### This section auto-scales the plot
x_min = min(DR.iloc[:,x_row])
x_max = max(DR.iloc[:,x_row])
y_min = min(DR.iloc[:,y_row])
y_max = max(DR.iloc[:,y_row])

dx = abs(x_min-x_max)
dy = abs(y_min-y_max)

x_min = x_min - abs(dx*0.05)
x_max = x_max + abs(dx*0.05)
y_min = y_min - abs(dy*0.05)
y_max = y_max + abs(dy*0.05)

plt.xlabel(f'{DR_sheet} 1',fontsize=18)
plt.ylabel(f'{DR_sheet} 2',fontsize=18)

plt.xlim(x_min, x_max)
plt.ylim(y_min, y_max)

if(DR_sheet == 'PCA'):
    plt.xticks(fontsize=18)
    plt.locator_params(axis='x', nbins=5)
    plt.yticks(fontsize=18)
    plt.locator_params(axis='y', nbins=4)
elif(DR_sheet == 'UMAP'):
    plt.xticks([])
    plt.yticks([])

# Make a list of ones the same length as in_hotspot_list
max_color = [1] * len(in_hotspot_list)

# Plot the ligands not in the hotspot
plt.scatter(DR.loc[not_in_hotspot_list, DR.columns[0]], DR.loc[not_in_hotspot_list, DR.columns[1]],c='white', edgecolor='black', s=100, alpha=0.7)
# Plot the ligands in the hotspot
plt.scatter(DR.loc[in_hotspot_list, DR.columns[0]], DR.loc[in_hotspot_list, DR.columns[1]], cmap='Oranges', c=max_color, edgecolor='black', s=100, alpha=0.7)  

# Include a legend for the colors
plt.clim(vmin=0, vmax=1)
plt.legend(['Not in Hotspot', 'In Hotspot'], fontsize=18)


plt.show() 